## Foto de Perfil

Vamos a agregar más elementos a la ruta de **`/profile/`**, comencemos con una foto de perfil.

Para esto vamos a generar una nueva tabla, ésta tabla tendrá una relación 1 a 1 con la table **`User`**, es decir, un usuario tendrá una foto de perfil y será una "extensión" de la clase **`User`**.

Como es una tabla debemos definir la clase en **`models.py`**:

```python
from django.db import models
from django.contrib.auth.models import User
from PIL import Image

class Profile(models.Model):
    user = models.OneToOneField(User, on_delete = models.CASCADE)
    image = models.ImageField(default = 'default.jpg', upload_to = 'profile_pics')
    
    def __str__(self):
        return f'{self.user.username} Profile'
    
    def save(self, *args, **kwargs):
        super().save(*args, **kwargs)
        
        img = Image.open(self.image.path)
        
        if img.height > 300 or img.width > 300:
            output_size = (300, 300)
            img.thumbnail(output_size)
            img.save(self.image.path)
        
        
```

**Nota:** el método **`save()`** es un método que hereda los parametros del método **`save()`** de **`models.Model`**. El código dentro de este método reduce el tamaño de las imagenes mayores a 300px de alto o ancho.

Para ver los cambios debemos ejecutar las lineas de **`migration`** en la consola:

```python
python manage.py makemigrations

python manage.py migrate
```

Veremos que nos da un error porque nos falta instalar una libreria llamada **`Pillow`**, podemos resolver este problema haciendo:
```python
pip install Pillow
```

**`Pillow`** es una libreria **`open-source`** de python que añade soporte para abrir, manipular, y guardar imagenes en diferentes formatos.

Y ahora ejecutamos las lineas de **`migrate`**.

No nos olvidemos que para ver esta tabla reflejada en la página de **`admin`** debemos añadirla desde el archivo **`admin.py`** de la app **`users`**:

```python
from django.contrib import admin
from .models import Profile

admin.site.register(Profile)
```

**Veamos la tabla en la página de `admin`.**


Vemos que la tabla se inicializó pero está vacia, por ahora vamos a dejarlo así y veamos como crear un perfil desde la página de **`admin`**.

Vamos a crear un perfil con imagen y otra que no tenga imagen para que tome la imagen por defecto.

Ahora quiero ver estos perfiles, vamos a hacerlo usando el **`shell`** de **`django`**. Vamos a la consola y ejecutamos:

```python
python manage.py shell
```
*Importamos la tabla User*

```python
from django.contrib.auth.models import User
```
*Y capturamos al usuario que tenga la foto de perfil en la variable user:*
```python
user = User.objects.filter(username="HackABoss").first()
```

_Como la tabla **`User`** tiene una relación **`OneToOne`** con la tabla **`Profile`** somos capaces de hacer:_

```python
user.profile

user.profile.image
```

_Esto último nos retorna un objeto **`ImageFieldFile`** con una ruta, si quisieramos esa ruta podemos hacer:_
```python
user.profile.image.url
```

Ahora veamos que pasa con el usuario que tiene la foto de perfil por defecto:

```python
user = User.objects.filter(username="HackABoss2").first()

user.profile.image.url
```

*Aun no hemos tenemos una imagen por defecto en esa ruta, por lo que este usuario no tiene imagen.*

```python
exit()
```

Si nos fijamos, dentro del proyecto tenemos ahora un nuevo directorio llamado **`profile_pics`** que ahora va a almacenar todas las fotos de perfil. Este es el nombre que pusimos en el parametro **`upload_to`** cuando definimos la clase **`Profile`**.

Imaginemos ahora que tenemos varias tablas o modulos que almacenan imagenes, para cada una tendremos una carpeta diferente, por lo que el directorio del proyecto se llenará de carpetas con imagenes, tal vez es mejor asignar una ubicación donde solo se almacenen esos directorios con imagenes, para esto podemos agregar al final del archivo **`settings.py`**:

```python
MEDIA_ROOT = Path(BASE_DIR, 'media')
MEDIA_URL = '/media/'
```

**`MEDIA_ROOT`** va a ser la ruta dentro de nuestro proyecto donde se almacenarán las carpetas con imagenes.

**`MEDIA_URL`** va a ser el enlace que usaremos para acceder a estas imagenes desde el navegador.

Con esto hecho, cada vez que tengamos una foto de perfil irá directo a esa ruta, pero las que ya estaban creadas no se verán afectadas por estos cambios, por lo que tendrémos que eliminar esos perfiles y crearlos otra vez. Y ya podemos eliminar el directorio **`profile_pics`**

Usando la página de **`admin`** eliminemos esos perfiles, e inmediatamente vamos a crear 2 perfiles, uno con foto de perfil y otro sin foto de perfil.

Si nos fijamos, se ha creado un directorio llamado **`media`** y dentro se ha creado otro llamado **`profile_pics`**, de ahora en adelante tendremos todo lo relacionado con imagenes en ese directorio.